<b>IMPORTING THE PYNQ OVERLAY
and configuring the HDMI and INPUT OUTPUT FRAME</b>

In [ ]:
from pynq import Overlay
Overlay("base.bit").download()

from pynq import Overlay
from pynq.lib.video import *

base = Overlay('base.bit')
#hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out

Mode = VideoMode(640,480,24)
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()

# monitor (output) frame buffer size
frame_out_w = 1920
frame_out_h = 1080
# camera (input) configuration
frame_in_w = 640
frame_in_h = 480

<b>CAPTURING THE REFRENCE FRAME AND CONVERTING IT TO GRAYSCALE</b>

In [ ]:
import cv2
import numpy as np

videoIn = cv2.VideoCapture(0)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);

print("capture device is open: " + str(videoIn.isOpened()))
#frame_1080p = np.zeros((1080,1920,3)).astype(np.uint8)
ret, reference = videoIn.read()
reference_grey = cv2.cvtColor(reference, cv2.COLOR_BGR2GRAY)
reference_blur = cv2.GaussianBlur(reference_grey, (5, 5), 0)

#frame_1080p[0:480,0:640,:] = reference[0:480,0:640,:]
#hdmi_out.writeframe(frame_1080p.astype(np.int8))

<b>NOW DETECTING THE MOTION IN REAL TIME USING OPENCV<b/>

In [ ]:
import numpy as np
import time
outframe = hdmi_out.newframe()
numframes = 2000    
start = time.time()    
for _ in range(numframes):
    ret, frame_vga = videoIn.read()

    if (ret):
       
        grey = cv2.cvtColor(frame_vga, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(grey, (5, 5), 0)
        difference = cv2.absdiff(reference_blur, blur)
        threshold = cv2.threshold(difference, 50, 255, cv2.THRESH_BINARY)[1]
        dilated = cv2.dilate(threshold, None, iterations=2)
        _, contours, hier = cv2.findContours(dilated.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for i in contours:
            if cv2.contourArea(i) < 2000:
                continue
            (x, y, w, h) = cv2.boundingRect(i)
           
            cv2.rectangle(frame_vga, (x, y), (x + w, y + h), (0, 0, 255), 2) 
            outframe[0:480,0:640,:] = frame_vga[0:480,0:640,:]
            hdmi_out.writeframe(outframe) 
end = time.time()
print("Frames per second:  " + str(numframes / (end - start)))


<b>RELEASING CAMERA AND SCREEN<b/>

In [ ]:
hdmi_out.close()
videoIn.release()